## KOUSHIK KANEPALLY - 017
## CH. VAMSHI - 010
## V NARAYANAN - 019

In [ ]:
import numpy as np
import random
import json
import nltk
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
nltk.download('punkt')




### Creating Custom Functions:
We will create custom Functions so that it is easy for us to implement afterwards.

In [ ]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)
#tokenizeit will divide sentence to words

def stem(word):
    return stemmer.stem(word.lower())


In [ ]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()


In [ ]:
##BAG OF WORDS

In [ ]:

def bag_of_words(tokenized_sentence, words):
    
    # stem each word
    sentence_words = [stem(word) for word in tokenized_sentence]
    # initialize bag with 0 for each word
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words: 
            bag[idx] = 1
    

    return bag

In [ ]:
with open('Intent.json', 'r') as f:
    intents = json.load(f)
intents



In order to get the right information, we will be unpacking it with the following code. 


In [ ]:
all_words = []
tags = []
xy = []
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    tag = intent['intent']
    # add to tag list
    tags.append(tag)
    for pattern in intent['text']:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag))



This will separate all the tags & words into their separate lists



### Cleaning & Preparing the data using our custom functions

We will be cleaning the data by implementing the functions we created in our previous cells.

In [ ]:
# stem and lower each word
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
# remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(xy), "patterns")
#143 rows
print(len(tags), "tags:", tags)
#22 classes ->22  output neurons in the final layer
print(len(all_words), "unique stemmed words:", all_words)
#114 input neurons in the input layer
xy


### Creating Training Data: 
We will transform the data into a format that our PyTorch Model can Easily Understand


In [ ]:
# create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)
y_train



    `One hot encoding Is the process of splitting multiclass or multi valued data column to separate columns and labelling the cell 1 in the row where it exists. (we won’t use it so don’t worry about it)`


In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size) 
        #input layer to hidden layer
        self.l2 = nn.Linear(hidden_size, hidden_size) 
        #hidden layer to hidden layer
        self.l3 = nn.Linear(hidden_size, num_classes)
        #hidden layer to output layer
        self.relu = nn.ReLU()
    
    def forward(self, x):
        #send to 1st hidden layer
        out = self.l1(x)
        #activate 1st hidden
        out = self.relu(out)
        #send to 2nd hidden
        out = self.l2(out)
        #activate second hidden
        out = self.relu(out)
          #send to output layer
        out = self.l3(out)

        return out

In [ ]:
class ChatDataset(Dataset):

    def __init__(self):
        #26 samples
        self.n_samples = len(X_train)
        #26 samples with 1 hort encoding
        self.x_data = X_train
        #26 smapoles with class number
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

In [ ]:
#26 rows
#8*3+2 =  1 epoch
#weight is getting updated after 8 rows
# Hyper-parameters 
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size, output_size)

In [ ]:
dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for s,y in train_loader:
  print(len(s[0]))
  print(y)


## Training the Model
---


In [ ]:

# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        # Forward pass
        outputs = model(words)
        
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


print(f'final loss: {loss.item():.4f}')

data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

In [ ]:

FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')



## Loading our Saved Model
---

In [ ]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('Intent.json', 'r') as json_data:
    intents = json.load(json_data)

FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()



 
## Using the Chatbot:


In [ ]:

bot_name = "Amrita Chatbot"
print("Let's chat! (type 'quit' to exit)")
while True:
    # sentence = "do you use credit cards?"
    sentence = input("You: ")
    if sentence == "quit":
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    print(X)
    X = X.reshape(1, X.shape[0])
    print(X)
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)
   
    tag = tags[predicted.item()]
    print(tag)
    probs = torch.softmax(output, dim=1)
    print(probs)
    prob = probs[0][predicted.item()]
    print(prob.item())
    if prob.item() > 0.75:
        for intent in intents['intents']:
            
            if tag == intent["intent"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand...")